In [1]:
using Pkg
Pkg.add(PackageSpec(name="JWAS", rev="master"))
Pkg.update()
Pkg.add("DataFrames")
Pkg.add("CSV")
using JWAS,JWAS.Datasets,DataFrames,CSV,Statistics


    Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
    Updating registry at `C:\Users\aisha\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`
    Updating registry at `C:\Users\aisha\.julia\registries\General.toml`
    Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`


In [2]:
# Step 2: Read data 
phenofile = dataset("C:/Users/aisha/Desktop/Grad_School/ROTATIONS/Jack_Lab/Stress_project/Analyses/SWIM_imputation/QTLmarkers/Hormones.csv")
genofile = dataset("C:/Users/aisha/Desktop/Grad_School/ROTATIONS/Jack_Lab/Stress_project/Analyses/GWAS/dummy_genotypes.txt")
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstrings=["NA"])
phenotypes = dropmissing(phenotypes, :qNurlogCort)
first(phenotypes,5)


┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV C:\Users\aisha\.julia\packages\CSV\aoJqo\src\context.jl:344


Row,ID,Batch,Company,SowID,EntryAge,Cortisol,Hair1_GrindingDate,C_ExtractionDate,C_GrindingToExtractionTime,qNurlogCort,SW_144836353,SW_144841591,SW_144792892,SW_144876147,SW_144827094,SW_144841166,qNurPenBatch,SW_144783524,SW_144783527,cNurlogCort,cNurPenBatch,GrindingDate2,cNur_groundstorage
,Int64,Int64?,String1?,String15?,Int64?,Float64?,String15?,String15?,Int64?,Float64,Int64,Int64,Int64,Int64,Int64,Int64,String7?,Int64,Int64,Float64?,String7?,String15?,Int64?
1,4935,81,E,E8B260,22,16.499,4/20/2022,5/5/2022,15,2.8033,0,0,0,0,0,0,81_1,0,0,2.14908,81_D02,29/08/2022,85
2,4936,81,E,W3W953,22,13.6,4/20/2022,5/5/2022,15,2.61007,0,0,0,0,0,0,81_1,0,0,1.757,81_B04,29/08/2022,85
3,4937,81,E,E8B205,17,23.1791,4/20/2022,5/5/2022,15,3.14325,0,0,0,0,0,0,81_1,0,0,2.40153,81_D08,29/08/2022,85
4,4938,81,E,E8B411,19,16.9478,6/9/2022,5/5/2022,missing,2.83014,1,0,1,0,1,1,81_1,1,1,-0.71744,81_B04,30/08/2022,84
5,4939,81,E,3240,16,13.7074,4/22/2022,6/13/2022,52,2.61794,0,0,0,0,0,0,81_1,0,0,1.46326,81_B06,29/08/2022,135


In [3]:
Hair_G5=0.151743E-01 #Additive var.
genotypes = get_genotypes(genofile,header=true,separator=',',Hair_G5,method="BayesB",quality_control=false, Pi = 0.999, estimatePi = false,estimate_scale=true);
#

The delimiterd in dummy_genotypes.txt is ','. The header (marker IDs) is provided in dummy_genotypes.txt.
Genotype informatin:
#markers: 4999; #individuals: 4095


In [4]:
# Step 3: Build Model Equations
Hair_model_equation = "qNurlogCort = Batch + EntryAge  + C_GrindingToExtractionTime + SowID + qNurPenBatch + C_ExtractionDate + Hair1_GrindingDate + genotypes";
#Hair_R=0.721038E-01; #Residual var
Hair_model = build_model(Hair_model_equation);#, Hair_R
#

In [5]:
# Step 4: Set Factors or Covariates
set_covariate(Hair_model, "EntryAge");
set_covariate(Hair_model, "C_GrindingToExtractionTime");

In [6]:
#Hair_G1=0.224905E-03;
set_random(Hair_model,"qNurPenBatch"); #,Hair_G1
#Hair_G2=0.257092E-01;
set_random(Hair_model,"SowID"); #,Hair_G2
#Hair_G3=0.05;
set_random(Hair_model,"C_ExtractionDate"); #,Hair_G3
#Hair_G4=0.05;
set_random(Hair_model,"Hair1_GrindingDate"); #,Hair_G4

In [7]:
# Step 5: Run Analysis
out=runMCMC(Hair_model,phenotypes, chain_length=200,burnin = 0,output_heritability=true, output_samples_frequency=10);


The folder results already exists.
The folder results1 already exists.
The folder results2 already exists.
The folder results3 already exists.
The folder results4 already exists.
The folder results5 already exists.
The folder results6 is created to save results.
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Predicted values for individuals of interest will be obtained as the summation of Any[] (Note that genomic data is always included for now).Missing values are found in column Batch for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. Users may impute missing values before the analysis 
Missing values are found in column EntryAge for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. Users may impute missing values before the ana

running MCMC ...   1%|█                                  |  ETA: 0:06:42

The file results6/MCMC_samples_heritability.txt is created to save MCMC samples for heritability.


running MCMC ... 100%|███████████████████████████████████| Time: 0:00:07




The version of Julia and Platform in use:

Julia Version 1.10.0
Commit 3120989f39 (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 16 × AMD Ryzen 7 5700U with Radeon Graphics         
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, znver2)
  Threads: 2 on 16 virtual cores


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




In [9]:
# Step 6: Run GWAS
cd("C:/Users/aisha/results6")
out = GWAS("MCMC_samples_marker_effects_genotypes_qNurlogCort.txt","dummy_mapfile.txt",Hair_model,window_size="1 Mb")
#sort(out, :prGenVar, rev=true)
#CSV.write("WindowEffectsqNurlogCort.txt",out)
pwd()


Compute the posterior probability of association of the genomic window that explains more than 0.001 of the total genetic variance.


running GWAS... 100%|████████████████████████████████████| Time: 0:00:02


"C:\\Users\\aisha\\results6"